# Pruebas Neo4j

In [1]:
from neo4j import GraphDatabase as GD
import pandas as pd
import numpy as np

In [2]:
uri = 'bolt://localhost:7687'
user = 'neo4j'
psw = 'password'

### Primera prueba

In [14]:
d = GD.driver(uri, auth=(user, psw))

In [15]:
def get_pod(tx, cat):
    pods = []
    result = tx.run("MATCH (c:Category)<--(p) "
                   "WHERE c.name = $cat "
                   "RETURN p.id AS pod", cat=cat)
    for r in result:
        pods.append(r['pod'])
    return pods

In [16]:
with d.session() as sess:
    pod = sess.read_transaction(get_pod, "news")
    for p in pod:
        print(p)

e89a2cc24f1fa1138a56a532ce68ab4a
f5fce0325ac6a4bf5e191d6608b95797
f42e0606b820b99354ab58d423598493
e8be02e7d7281ba3edb77da6a9fe95cd
ebb2a803a79ea0b5abdefb92d1c816f5
fc32aac8fe942ed89feae0e164185912
c4eafea85540de95544b73777e19c696
b9a5920d121ff4e239e6ca3b210e65e7
a343c93505bffeaf20564bd2977b4743
f1a7cabd6127108563a0e38ae044b88a
fa7162cfb3963c4d51b67094a51740d6
c684ea6c24f8a6a16c18edd37f2aea41
fa3cac03e05c1e9ec792e95e4850d7ce


### Segunda Prueba
Generar una clase

In [61]:
class App:
    def __init__(self, uri, auth):
        self.driver = GD.driver(uri, auth = auth)
        
    def close(self):
        self.driver.close()
        
    def get_cat_pod(self, user_id):
        with self.driver.session() as sess:
            result = sess.write_transaction(
                self._get_cat_pod, user_id)
            return result
        
    @staticmethod
    def _get_cat_pod(tx, user_id):
        query = (
            "MATCH (u:User)-[r]->(Podcast)-->(Category)<--(p:Podcast) "
            "WHERE u.id = $user_id "
            "RETURN r, p "
        )
        result = tx.run(query, user_id=user_id)
        return pd.DataFrame([{'cum_rating':record['r']['rating'], 
                              'podcast_id':record['p']['id']}
                    for record in result])

In [62]:
app = App(uri, (user, psw))

In [63]:
df = app.get_cat_pod("F5B4FD0698E670E")
df.head()

,cum_rating,podcast_id
0,5,bead83f2330788f9e629cce951e1df99
1,5,cc451ec20dfe8f985c246cc71749917d
2,5,b16332f51b9746dc5a97c7ce37644fc6
3,5,c1adb6ca5ca39575420fda03c099b037
4,5,cc451ec20dfe8f985c246cc71749917d


In [64]:
df.groupby('podcast_id')['cum_rating'].sum().reset_index()

,podcast_id,cum_rating
0,a3a535f66c7e8004e7dc54c2b2829a9e,5
1,b16332f51b9746dc5a97c7ce37644fc6,55
2,bead83f2330788f9e629cce951e1df99,55
3,c1adb6ca5ca39575420fda03c099b037,55
4,c2adb71e6b51e1397b49e216e07f3cef,55
5,cc451ec20dfe8f985c246cc71749917d,555
6,ee41bdc529ac60b6cd3b9a50413b0dee,5
7,f335df99185d1401ada492230b42514a,5
8,fa3cac03e05c1e9ec792e95e4850d7ce,5


## Generar atributos para entrenar modelo

In [137]:
class GenAtr:
    def __init__(self, uri, auth):
        self.driver = GD.driver(uri, auth = auth)
        
    def close(self):
        self.driver.close()
        
    
    def gen_train_data(self, df):
        c_names = ['cat_based', 'cat_cnt', 'user_based', 'user_cnt',
                   'adamic_adar', 'resource_allocation', 'link_cnt']
        df[c_names] = df.apply(self.gen_train_data_row, axis=1, result_type='expand')
        return df 
    
    def gen_train_data_row(self, row):
        u_id, p_id, rtg = row['user_id'], row['podcast_id'], row['rating']
        self.delete_rtg(u_id, p_id)
        
        result = self.get_cat_based(u_id, p_id)
        result += self.get_user_based(u_id, p_id)
        result += self.adamic_adar(u_id, p_id)
        result += self.resource_allocation(u_id, p_id)
        
        self.create_rtg(u_id, p_id, rtg)
        
        return result
        
    def delete_rtg(self, user_id, podcast_id):
        with self.driver.session() as sess:
            sess.write_transaction(
                self._delete_rtg, user_id, podcast_id)
    
    @staticmethod
    def _delete_rtg(tx, user_id, podcast_id):
        query = (
            "MATCH (u:User)-[r]->(p:Podcast) "
            "WHERE u.id = $user_id AND p.id = $podcast_id "
            "DELETE r"
        )
        tx.run(query, user_id=user_id, podcast_id=podcast_id)

        
    def create_rtg(self, user_id, podcast_id, rating):
        with self.driver.session() as sess:
            sess.write_transaction(
                self._create_rtg, user_id, podcast_id, rating)
    
    @staticmethod
    def _create_rtg(tx, user_id, podcast_id, rating):
        query = (
            "MATCH (u:User) MATCH (p:Podcast) "
            "WHERE u.id = $user_id AND p.id = $podcast_id "
            "MERGE (u)-[r:Rating{rating:toInteger($rating)}]->(p) "
        )
        tx.run(query, user_id=user_id, podcast_id=podcast_id, rating=rating)
        
    def get_cat_based(self, user_id, podcast_id):
        with self.driver.session() as sess:
            result = sess.write_transaction(
                self._get_cat_based, user_id, podcast_id)
            return result
        
    @staticmethod
    def _get_cat_based(tx, user_id, podcast_id):
        query = (
            "MATCH (u:User)-[r]->(Podcast)-->(Category)<--(p:Podcast) "
            "WHERE u.id = $user_id AND p.id = $podcast_id "
            "RETURN r"
        )
        result = tx.run(query, user_id=user_id, podcast_id=podcast_id)
        total = 0
        cnt = 0
        for rec in result:
            total += rec['r']['rating']
            cnt += 1
        return [total, cnt]
    
    def get_user_based(self, user_id, podcast_id):
        with self.driver.session() as sess:
            result = sess.write_transaction(
                self._get_user_based, user_id, podcast_id)
            return result
        
    @staticmethod
    def _get_user_based(tx, user_id, podcast_id):
        query = (
            "MATCH (u:User)-[r1]->(Podcast)<-[r2]->(User)-[r3]->(p:Podcast) "
            "WHERE u.id = $user_id AND p.id = $podcast_id "
            "RETURN r1.rating + r2.rating + r3.rating "
            "AS total"
        )
        result = tx.run(query, user_id=user_id, podcast_id=podcast_id)
        total = 0
        cnt = 0
        for rec in result:
            total += rec['total']
            cnt += 1
        return [total, cnt]
    
    def adamic_adar(self, user_id, podcast_id):
        with self.driver.session() as sess:
            result = sess.write_transaction(
                self._adamic_adar, user_id, podcast_id)
            return result
        
    @staticmethod
    def _adamic_adar(tx, user_id, podcast_id):
        query = (
            "MATCH (u:User)-[r]->(p1:Podcast) MATCH (p:Podcast)"
            "WHERE u.id = $user_id AND p.id = $podcast_id "
            "RETURN r.rating * gds.alpha.linkprediction.adamicAdar(p1, p) "
            "AS score "
        )
        result = tx.run(query, user_id=user_id, podcast_id=podcast_id)
        total = 0
        for rec in result:
            total += rec['score']
        return [total]
    
    def resource_allocation(self, user_id, podcast_id):
        with self.driver.session() as sess:
            result = sess.write_transaction(
                self._resource_allocation, user_id, podcast_id)
            return result
        
    @staticmethod
    def _resource_allocation(tx, user_id, podcast_id):
        query = (
            "MATCH (u:User)-[r]->(p1:Podcast) MATCH (p:Podcast)"
            "WHERE u.id = $user_id AND p.id = $podcast_id "
            "RETURN r.rating * gds.alpha.linkprediction.resourceAllocation(p1, p) "
            "AS score "
        )
        result = tx.run(query, user_id=user_id, podcast_id=podcast_id)
        total = 0
        cnt = 0
        for rec in result:
            total += rec['score']
            cnt += 1
        return [total, cnt]

In [138]:
gen = GenAtr(uri, (user, psw))

Generación de atributos

In [139]:
rtg = pd.read_csv('data/ratings_sample.csv')
rtg.head()

,podcast_id,rating,user_id,liked
0,d34629baca14d87860276b12a6cb7a3b,1,949410AE9D72DA4,0
1,d34629baca14d87860276b12a6cb7a3b,5,F9939C4F2B7BCA4,1
2,a3a535f66c7e8004e7dc54c2b2829a9e,5,96A3F0358E7808D,1
3,a6cc4101cb004140936103e2ffbf8451,5,96A3F0358E7808D,1
4,a6cc4101cb004140936103e2ffbf8451,5,F5B4FD0698E670E,1


In [142]:
train_sample = gen.gen_train_data(rtg[:10].copy())
train_sample['cat_avg'] = train_sample['cat_based'] / train_sample['cat_cnt']
train_sample['user_avg'] = train_sample['user_based'] / train_sample['user_cnt']
train_sample['adar_avg'] = train_sample['adamic_adar'] / train_sample['link_cnt']
train_sample['ra_avg'] = train_sample['resource_allocation'] / train_sample['link_cnt']
train_sample = train_sample.fillna(0)
train_sample.fillna(0).head()

,podcast_id,rating,user_id,liked,cat_based,cat_cnt,user_based,user_cnt,adamic_adar,resource_allocation,link_cnt,cat_avg,user_avg,adar_avg,ra_avg
0,d34629baca14d87860276b12a6cb7a3b,1,949410AE9D72DA4,0,1.0,1.0,3.0,1.0,0.835049,0.193694,2.0,1.0,3.0,0.417524,0.096847
1,d34629baca14d87860276b12a6cb7a3b,5,F9939C4F2B7BCA4,1,0.0,0.0,0.0,0.0,0.000000,0.000000,4.0,0.0,0.0,0.000000,0.000000
2,a3a535f66c7e8004e7dc54c2b2829a9e,5,96A3F0358E7808D,1,25.0,5.0,210.0,14.0,40.093175,9.035088,10.0,5.0,15.0,4.009317,0.903509
3,a6cc4101cb004140936103e2ffbf8451,5,96A3F0358E7808D,1,40.0,8.0,135.0,9.0,36.960622,8.618421,10.0,5.0,15.0,3.696062,0.861842
4,a6cc4101cb004140936103e2ffbf8451,5,F5B4FD0698E670E,1,0.0,0.0,0.0,0.0,0.000000,0.000000,0.0,0.0,0.0,0.000000,0.000000


In [143]:
train_sample.describe()

,rating,liked,cat_based,cat_cnt,user_based,user_cnt,adamic_adar,resource_allocation,link_cnt,cat_avg,user_avg,adar_avg,ra_avg
count,10.000000,10.000000,10.000000,10.000000,10.000000,10.000000,10.000000,10.000000,10.000000,10.00000,10.000000,10.000000,10.000000
mean,4.600000,0.900000,13.100000,2.700000,72.300000,4.900000,16.343826,3.803859,4.700000,2.60000,7.800000,2.050882,0.474043
std,1.264911,0.316228,16.278479,3.198958,91.339477,6.026792,20.076927,4.682334,4.715224,2.54733,7.641989,2.082559,0.481202
min,1.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.00000,0.000000,0.000000,0.000000
25%,5.000000,1.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.250000,0.00000,0.000000,0.000000,0.000000
50%,5.000000,1.000000,3.000000,1.000000,9.000000,1.000000,2.545841,0.574120,3.000000,3.00000,9.000000,2.056793,0.479344
75%,5.000000,1.000000,25.000000,5.000000,168.750000,11.250000,38.915581,8.930921,10.000000,5.00000,15.000000,3.996169,0.928509
max,5.000000,1.000000,40.000000,8.000000,210.000000,14.000000,41.725544,9.868421,10.000000,5.00000,15.000000,4.256634,0.986842
